In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TextDataset, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import jsonlines
import re
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer

C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def read_jsonl_to_dataframe(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            data.append(line)
    df = pd.DataFrame(data)
    return df

In [4]:
train_df = read_jsonl_to_dataframe('arabic_train.jsonl')
val_df = read_jsonl_to_dataframe('arabic_val.jsonl')
test_df = read_jsonl_to_dataframe('arabic_test.jsonl')

In [5]:
def summarize_arabic_text(text, summary_length=3):
    # Step 1: Tokenize the text into sentences
    nltk.download('punkt')
    sentences = sent_tokenize(text)

    # Step 2: Calculate sentence similarity using TF-IDF
    vectorizer = CountVectorizer()
    sentence_vectors = vectorizer.fit_transform(sentences)
    tfidf_transformer = TfidfTransformer()
    tfidf_matrix = tfidf_transformer.fit_transform(sentence_vectors)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Step 3: Select the most important sentences based on similarity
    sentence_scores = [(i, sum(similarity_matrix[i])) for i in range(len(sentences))]
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
    selected_sentences = [sentences[i] for (i, score) in sentence_scores[:summary_length]]

    # Step 4: Combine the selected sentences to create the summary
    summary = ' '.join(selected_sentences)

    return summary

In [ ]:
with jsonlines.open('validation.jsonl', mode='w') as writer:

    # Iterate through each row in the dataset and generate summaries
    for index, row in val_df.iterrows():
        summary = row['summary']
        # example_id = row['example_id']
        arabic_text = row['text']
        predicted = summarize_arabic_text(arabic_text, summary_length=2)

        # Create a dictionary for the summary
        summary_dict = {

            'summary': summary,
            'predicted': predicted
        }

        # Append the summary dictionary to the list
        writer.write(summary_dict)

        print(f"{summary}")

In [90]:

train = read_jsonl_to_dataframe('train.jsonl')
validation = read_jsonl_to_dataframe('validation.jsonl')
test = read_jsonl_to_dataframe('test.jsonl')

In [63]:
train

,summary,predicted
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,تسيطر قوات روسية حاليا على معظم القواعد العسكر...
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...","وأضاف لبي بي سي ""كان من المفترض أن تتم عملية ت..."
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,الجنة الملعونة: دموع الفقراء في كشمير لماذا يل...
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ومنذ أن حط كوكاي، الذي عرف بعد وفاته باسم كوبو...
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",كما أكد المصدر تمكن مسلحي الحراك من اعتقال مسل...
...,...,...
37514,*هذا الفيديو بدون تعليق*,ويقذف البركان بأنهار من الحمم البركانية عبر ال...
37515,بي بي سي ترند تقرير مي نعمان,لكن القليل من المغردين رأوا ان في هذه الحملة ن...
37516,تداعيات متواصلة,رمضان آخر يستقبله المسلمون في ظل كورونا للعام ...
37517,حملة شامتة,ومع وصول ذاك الجدل، الذي تحدث عنه تسيدال إلى ذ...


In [12]:
test

,summary,predicted
0,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,وسيواجه حسن عددا من ضحاياه في قاعة المحكمة لأن...
1,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,قال التليفزيون السوري إن قوات الحكومة استعادت ...,وكان الجيش السوري قد استعاد قبل نحو شهرين السي...
3,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,ويعتقد بأن ما بين 250 ألف إلى 400 ألف من الشبك...
...,...,...
4684,مجموعة مختارة من أفضل الصور في القارة الأفريقي...,رجل سوداني يغادر مخبزا وهو محمل بحقيبة مملوءة ...
4685,استأنف الرئيس الفلسطيني محمود عباس ورئيس الوزر...,"كما شددت على ان بلادها ""ستكون شريكا دائما ونشي..."
4686,هناك الكثير مما يجمع كارول نحاس بزوجها كارلوس ...,الدفاع عن كارلوس بعد هروب كارلوس غصن من إقامته...
4687,افرجت السلطات السودانية عن 57 من معتقلي حركة ا...,دعوة ودعا زعيم الحركة خليل ابراهيم بقية الجماع...


In [13]:
validation

,summary,predicted
0,وقع الحوثيون اتفاقا مع عدد من الاحزاب اليمنية ...,دخل القتال بين القوات الحكومية والمسلحين الحوث...
1,ناقش معلقون في صحف عربية تطور الأزمة السياسية ...,"""معركة"" الحكومة والاتحاد العام للشغل في سياقٍ ..."
2,اتهمت الشرطة البريطانية رسميا رجلا فرنسيا بحيا...,"أعلنت حالة التأهب الأمني بسبب ""تصرفات مشبوهة"" ..."
3,تطلب شابة على إحدى صفحات فيسبوك المغلقة نصيحة ...,"وتتفق عليا خيربك سليمان، مؤسسة مجموعة ""صبايا"" ..."
4,فرضت الولايات المتحدة عقوبات على بنك العملة ال...,قطعت كوريا الشمالية الخط الهاتفي الساخن بين ال...
...,...,...
4684,في الأول من مارس/ آذار الجاري بدأ نحو 27 ألف ج...,استولى تنظيم الدولة الإسلامية على الموصل في يو...
4685,تسود حالة من الهدوء الحذر محيط القصر والشوارع ...,وكانت قوات الأمن، التي كانت متمركزة قريبا من ن...
4686,حذر رئيس حزب العمال الكردستاني السجين، عبدالله...,أدى تقدم مسلحو الدولة الإسلامية تجاه كوباني إل...
4687,تناول بعض الأنواع الشائعة من مسكنات الألم له ع...,وأشار إلى أن هذه المسكنات يشيع استخدامها من جا...


In [28]:
# def remove_features(dataset):
#     features_to_remove = ['example_id']
#     for feature in features_to_remove:
#         if feature in dataset:
#             del dataset[feature]
#     return dataset


In [29]:
# merged_df1 = remove_features(merged_df1)
# merged_df2 = remove_features(merged_df2)


In [14]:
def get_max_word_count(dataset):
    max_word_count = 0
    for row in dataset:
        words = row.split()  # Splitting the text into words
        word_count = len(words)
        if word_count > max_word_count:
            max_word_count = word_count
    return max_word_count



In [15]:
max_word_count_p = get_max_word_count(train['predicted'])
print("Maximum word count paragraph:", max_word_count_p)
max_word_count_s = get_max_word_count(train['summary'])
print("Maximum word count summary:", max_word_count_s)

Maximum word count paragraph: 515
Maximum word count summary: 236


In [16]:
max_word_count_p = get_max_word_count(test['predicted'])
print("Maximum word count paragraph:", max_word_count_p)
max_word_count_s = get_max_word_count(test['summary'])
print("Maximum word count summary:", max_word_count_s)

Maximum word count paragraph: 352
Maximum word count summary: 47


In [17]:
max_word_count_p = get_max_word_count(validation['predicted'])
print("Maximum word count paragraph:", max_word_count_p)
max_word_count_s = get_max_word_count(validation['summary'])
print("Maximum word count summary:", max_word_count_s)

Maximum word count paragraph: 253
Maximum word count summary: 45


In [18]:

tokenizer = GPT2Tokenizer.from_pretrained("aubmindlab/aragpt2-base")
model = GPT2LMHeadModel.from_pretrained("aubmindlab/aragpt2-base")
tokenizer.pad_token = tokenizer.eos_token

In [91]:

from datasets import Dataset
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)
validation_dataset = Dataset.from_dict(validation)

In [65]:
train_dataset

Dataset({
    features: ['summary', 'predicted'],
    num_rows: 37519
})

In [21]:
test_dataset

Dataset({
    features: ['summary', 'predicted'],
    num_rows: 4689
})

In [22]:
validation_dataset

Dataset({
    features: ['summary', 'predicted'],
    num_rows: 4689
})

In [88]:
import numpy as np

def tokenize(example):
    input_text = example["predicted"]
    target_text = example["summary"]

    input_tokens = tokenizer(input_text, truncation=True, padding="max_length", max_length=515)
    target_tokens = tokenizer(target_text, truncation=True, padding="max_length", max_length=515)

    return {
        "input_ids": np.array(input_tokens["input_ids"]),
        "attention_mask": np.array(input_tokens["attention_mask"]),
        "labels": np.array(target_tokens["input_ids"]),
    }



In [92]:

train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["predicted", "summary"])
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=["predicted", "summary"])
validation_dataset = validation_dataset.map(tokenize, batched=True, remove_columns=["predicted", "summary"])

In [93]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 37519
})

In [68]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4689
})

In [94]:

from transformers import TrainingArguments
from sklearn.metrics import accuracy_score

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=10,
    # per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
    save_strategy='epoch'
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)


  0%|          | 0/46900 [05:05<?, ?it/s]


In [ ]:
trainer.train()

In [ ]:
# input_text = "انا ادري مين انا يا حبيبي"
input_text="artist: ويجز genre: 4"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
output = model.generate(input_ids, max_length=128)
# print(tokenizer.batch_decode(output[0]))
for i, generated_text in enumerate(tokenizer.batch_decode(output)):
    print(f"Generated text {i + 1}:")
    print(generated_text)


In [ ]:
def evaluate_summary(y_test, predicted):
    rouge_score = rouge.Rouge()
    scores = rouge_score.get_scores(y_test, predicted, avg=True)
    score_1 = round(scores['rouge-1']['f'], 2)
    score_2 = round(scores['rouge-2']['f'], 2)
    score_L = round(scores['rouge-l']['f'], 2)
    print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:", score_L,
          "--> avg rouge:", round(np.mean([score_1, score_2, score_L]), 2))